**Image Recognition using Various Known Predefined Models**
For this purpose of study I have selected to model which are widely popular and known as well.
- TensorFlow Kera
- Pytorch (using YOLO)
Both of Them are good and well implemented model which can be used for faster processing and development.

But **TensorFlow Kera** has found out to be better model than the YOLO.

Below are Implenetation code for both of the algorithms.

**TensorFlow**

In [ ]:
!pip install tensorflow keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import tensorflow as tf
model = tf.keras.applications.MobileNetV2(weights='imagenet')
imgp = '/content/car.jpeg'
image = tf.keras.preprocessing.image.load_img(imgp, target_size=(224, 224))
input_image = tf.keras.preprocessing.image.img_to_array(image)
input_image = tf.keras.applications.mobilenet_v2.preprocess_input(input_image)
input_image = tf.expand_dims(input_image, axis=0)
predictions = model.predict(input_image)
predicted_classes = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=5)[0]

1/1 [==============================] - 1s 885ms/step


In [40]:
for class_id, class_name, probability in predicted_classes:
    print(f"{class_name} ({class_id}): {round(probability*100, 2)}%")

sports_car (n04285008): 50.75%
racer (n04037443): 38.15%
convertible (n03100240): 5.34%
grille (n03459775): 1.21%
car_wheel (n02974003): 0.88%


**YOLO**

In [2]:
!pip install Pillow

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00


In [4]:
!pip install torch

In [5]:
!pip install requests

In [7]:
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image
import torch
import requests

In [44]:
url = 'https://www.ford.com/cmslibs/content/dam/vdm_ford/live/en_us/ford/nameplate/mustang/2022/collections/equipment/3-2/22_FRD_MST_55069_GT500_3Q_Front_Passenger_BLUE_DC_1_32.jpg'
image = Image.open(requests.get(url, stream=True).raw)

In [45]:
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

In [46]:
inputs = image_processor(images = image, return_tensors = "pt")
outputs = model(**inputs)

In [47]:
logits = outputs.logits
bboxes = outputs.pred_boxes

In [48]:
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes = target_sizes)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

Detected car with confidence 0.997 at location [344.32, 462.07, 1806.67, 1134.61]
